In [1]:
%load_ext autoreload

In [2]:
from pathlib import Path
import numpy as np
import json
from split_dataset import SplitDataset
from fimpy.pipeline.alignment import align_2p_volume
%autoreload

ModuleNotFoundError: No module named 'fimpy'

In [3]:
import tifffile as tiff
from tqdm import tqdm
from shutil import copy

In [6]:
def restructure_for_suite2p(path, ds_name="original"):
    ds_path = path / ds_name

    suite_2p_dir = (path / "suite2p")
    suite_2p_dir.mkdir(exist_ok=True)
    for f in tqdm(list(ds_path.glob("*.h5"))):
        dest_dir = suite_2p_dir / f.stem
        dest_dir.mkdir(exist_ok=True)
        copy(f,dest_dir / f.name)

In [7]:
source_dir = "anatomy\original"
source_dir = "original"
frames_per_plane = None
force_realign = False

In [8]:
master = Path(r"")
all_fish = list(master.glob("*_f*"))

In [9]:
for f in all_fish:
    print(f)
    try:
        if not(f / 'aligned').exists() or force_realign:
            align_2p_volume(SplitDataset(f / source_dir), output_dir=str(f), across_planes=False)
        else:
            print("Already aligned")

        if not(f / 'anatomy.tif').exists() or force_remake_tifs:
            ds = SplitDataset(f / "aligned")
            stack = ds[:, :, :, :]
            mean = stack.mean(0)
            tiff.imsave(str(f / "anatomy.tif"), mean)

        else:
            print("Already anatomized")
    except:
        print("Error during alignment or anatomy")

In [10]:
for f in all_fish:
    if not (f / "suite2p").exists():
        print(f)
        restructure_for_suite2p(f)